In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark_app = SparkSession.builder.appName('SparkSQL').getOrCreate()

In [ ]:
import pandas as pd

pandas_df = pd.read_csv("/content/CarFuelandEmissions2000_2013.csv", on_bad_lines='skip', sep=',', engine='python')
pandas_df.fillna(0, inplace=True)

df = spark_app.createDataFrame(pandas_df)
df.show()

+--------------------+----+------------+--------------------+------------------+-------------+--------+------------+-----------------+---------------+---------+------------+------------------+---------------+--------------+--------------------+-----------------+-----------+---+-------------+------------+-------------+-----------------+----------------------+---------------------+--------------------+------------------+-----------------+--------------------+-------------------+--------------+
|                file|year|manufacturer|               model|       description|euro_standard|tax_band|transmission|transmission_type|engine_capacity|fuel_type|urban_metric|extra_urban_metric|combined_metric|urban_imperial|extra_urban_imperial|combined_imperial|noise_level|co2|thc_emissions|co_emissions|nox_emissions|thc_nox_emissions|particulates_emissions|fuel_cost_12000_miles|fuel_cost_6000_miles|standard_12_months|standard_6_months|first_year_12_months|first_year_6_months|date_of_change|
+-----

In [ ]:
# Emissão total de monoxido de carbono
df.select(sum("co_emissions")).show()

+--------------------+
|   sum(co_emissions)|
+--------------------+
|1.6950839600999996E7|
+--------------------+



In [ ]:
# Medida de barulho dos carros manuais
df.where(df["transmission_type"] == "Manual").select("noise_level").agg({"noise_level": "avg"}).show()

+----------------+
|avg(noise_level)|
+----------------+
|72.4058903700387|
+----------------+



In [ ]:
# Contagem de modelos dos carros
df.groupBy("model").count().orderBy("model", ascending=True).show()

+--------------------+-----+
|               model|count|
+--------------------+-----+
|  08MY Chrysler 300C|    6|
|08MY Chrysler PT ...|    5|
|08MY Chrysler Seb...|    3|
|08MY Chrysler Voy...|    2|
|08MY Jeep Cheroke...|    2|
| 08MY Jeep Commander|    2|
|08MY Jeep Grand C...|    3|
|  08MY Jeep Wrangler|    5|
|  09MY Chrysler 300C|    4|
|09MY Chrysler Gra...|    2|
|09MY Chrysler Seb...|    7|
|  09MY Jeep Cherokee|    2|
| 09MY Jeep Commander|    1|
|   09MY Jeep Compass|    3|
|09MY Jeep Grand C...|    2|
|  09MY Jeep Wrangler|    5|
|1 Series 3-door F...|   77|
|1 Series 5-door F...|   33|
|1 Series 5-door F...|   82|
|1 Series E81/E82/...|  291|
+--------------------+-----+
only showing top 20 rows



In [ ]:
# Media Km/L combinado (Cidade e Rodovia)
df.select("combined_metric").agg({"combined_metric": "avg"}).show()

+--------------------+
|avg(combined_metric)|
+--------------------+
|   7.720775197205011|
+--------------------+



In [ ]:
# Tipo de combustível por preço (preço por 12000 milhas e 6000 milhas)
df.groupBy("fuel_type").agg({"fuel_cost_12000_miles": "avg", "fuel_cost_6000_miles": "avg"}).show()

+--------------------+--------------------------+-------------------------+
|           fuel_type|avg(fuel_cost_12000_miles)|avg(fuel_cost_6000_miles)|
+--------------------+--------------------------+-------------------------+
|              Diesel|        1118.1072178297404|        86.60656583018219|
|     Petrol Electric|        1427.4117647058824|                      0.0|
|                 CNG|         70.35483870967742|                      0.0|
|       Petrol Hybrid|        1135.9236641221373|       2.7022900763358777|
|                 LPG|        277.89115646258506|       238.69387755102042|
|        LPG / Petrol|         561.1538461538462|                      0.0|
|              Petrol|        1134.8623615362164|       215.23531555166886|
|         Electricity|        408.14285714285717|                      0.0|
|Petrol / E85 (Fle...|        2024.1176470588234|                      0.0|
|     Diesel Electric|        1129.3636363636363|                      0.0|
|        Pet

In [ ]:
# Taxa de imposto do veiculo (modelo, taxa de imposto padrão para 6 e 12 meses e para os primeiros 6 e 12 meses)
df.select("model", "standard_6_months", "standard_12_months", "first_year_6_months", "first_year_12_months").distinct().orderBy("first_year_6_months", ascending=False).show()

+--------------------+-----------------+------------------+-------------------+--------------------+
|               model|standard_6_months|standard_12_months|first_year_6_months|first_year_12_months|
+--------------------+-----------------+------------------+-------------------+--------------------+
|   C30 Model Year 13|             93.5|             170.0|               93.5|               170.0|
| XV AWD From 2012 MY|             93.5|             170.0|               93.5|               170.0|
|    BRZ From 2012 MY|             93.5|             170.0|               93.5|               170.0|
|              207 SW|             93.5|             170.0|               93.5|               170.0|
|               Venga|             93.5|             170.0|               93.5|               170.0|
|MINI Roadster R59...|             93.5|             170.0|               93.5|               170.0|
|Insignia 5 Door S...|             93.5|             170.0|               93.5|            

In [ ]:
# Carro mais econômico por tipo de combustível
df.groupBy("fuel_type").agg(max_by("model", "combined_metric").alias("model"), max("combined_metric").alias("combined_metric")).show()

+--------------------+--------------------+---------------+
|           fuel_type|               model|combined_metric|
+--------------------+--------------------+---------------+
|                 CNG| S80 Model Year 2002|           10.5|
|              Diesel|B-Class, Model Ye...|           44.0|
|     Diesel Electric|E-Class Estate, M...|            4.5|
|         Electricity|                Leaf|            0.0|
|  Electricity/Diesel|V60, Model Year 2014|            1.8|
|  Electricity/Petrol|               Prius|            2.1|
|                 LPG|        Shogun Sport|           16.8|
|        LPG / Petrol|      Zafira, MY2005|           11.4|
|              Petrol|Diablo Model Year...|           25.4|
|        Petrol / E85|9-5 MY2009 5 Door...|           10.6|
|Petrol / E85 (Fle...|         Continental|           17.0|
|     Petrol Electric|5 Series F10/F11,...|            7.0|
|       Petrol Hybrid|           LS MY2011|            9.3|
+--------------------+------------------

In [ ]:
# Média co2 emitido por marca
df.groupBy("manufacturer").agg({"co2": "avg"}).orderBy("avg(co2)", ascending=False).show()

+--------------------+------------------+
|        manufacturer|          avg(co2)|
+--------------------+------------------+
|         Lamborghini| 476.2135922330097|
|      Bentley Motors|             419.5|
|             Ferrari| 417.4567901234568|
|         Rolls-Royce| 393.7313432835821|
|            Maserati| 382.2068965517241|
|Aston Martin Lagonda| 371.6190476190476|
|              Hummer|             336.5|
|            Corvette|316.09615384615387|
|               Isuzu| 308.4230769230769|
|             McLaren|             279.0|
|          Land Rover| 273.0662251655629|
|             Porsche| 262.6155115511551|
|            Metrocab|258.09090909090907|
|                Tata|255.57142857142858|
|            Cadillac|254.77906976744185|
|            Infiniti| 253.5744680851064|
|         Jaguar Cars|249.27373068432672|
|       Chrysler Jeep|248.18688524590164|
|               Lotus|241.22471910112358|
|           SsangYong|230.64880952380952|
+--------------------+------------

In [ ]:
# A maior contagem de Euro Standard
euro_standard = df.groupBy("euro_standard").count().orderBy("count", ascending=False)
euro_standard.show()

+-------------+-----+
|euro_standard|count|
+-------------+-----+
|            4|20269|
|            5|12271|
|            3|10717|
|            2| 1865|
|            6|  389|
+-------------+-----+



In [ ]:
# Média de variação de emissão de CO2 pro ano por marca

from pyspark.sql.window import Window

manufacturer_year = df.groupBy("manufacturer", "year").agg({"co2": "avg"})
partition = Window.partitionBy("manufacturer").orderBy("year")
co2_variations = manufacturer_year.withColumn("previous", lag("avg(co2)", 1, 0).over(partition)) \
.filter(col("previous") != 0).withColumn("co2_variation", col("previous") - col("avg(co2)"))
media_variacao = co2_variations.groupBy("manufacturer").agg({"co2_variation": "avg"}).orderBy("avg(co2_variation)", ascending=False)
media_variacao.show()


+--------------------+------------------+
|        manufacturer|avg(co2_variation)|
+--------------------+------------------+
|         Lamborghini|13.290909090909095|
|           SsangYong|12.911111111111113|
|      Bentley Motors|12.591666666666669|
|Aston Martin Lagonda|12.258461538461535|
|           Chevrolet|11.836884555194414|
|             Ferrari|11.207692307692307|
|            Cadillac|11.136200716845877|
|         Daewoo Cars| 9.793939393939393|
|          Land Rover| 8.383484162895929|
|          Alfa Romeo| 8.199095022624435|
|               Lexus| 8.008461538461539|
|                 Kia|7.4892906494702896|
|       Mercedes-Benz| 6.905844486639189|
|                 BMW| 6.744564631536535|
|         Jaguar Cars| 6.422485207100592|
|            Maserati| 6.291666666666668|
|              Toyota| 6.215841236389182|
|         Rolls-Royce| 6.102564102564101|
|               Dodge| 5.908333333333331|
|       Chrysler Jeep| 5.868180882453855|
+--------------------+------------